In [87]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn import metrics
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
import collections

In [189]:
test = pd.read_csv("test1_combined.csv")
df = pd.read_csv("train1_combined.csv")
dev = pd.read_csv("dev1_combined.csv")
#test.drop(test.columns[[13,14]], axis=1, inplace=True)
#df.drop(df.columns[[13,14]], axis=1, inplace=True)

In [190]:
x_train = df.iloc[:,4:-1]
y_train = df.label
x_dev = dev.iloc[:,4:-1]
y_dev = dev.label

In [72]:
df.shape

(27358, 11)

In [73]:
dev.shape

(10110, 11)

In [74]:
len(dev[dev.label ==0])

5055

In [75]:
df.iloc[:,4:-1]

,aa,ra,jc,pa,dc_A,dc_B
0,2.508376,0.551129,0.461538,88,0.002317,0.003187
1,3.049436,0.707973,0.500000,104,0.002317,0.003766
2,2.984187,0.673785,0.700000,72,0.002317,0.002607
3,2.027478,0.426129,0.500000,56,0.002317,0.002028
4,3.060384,0.713467,0.466667,112,0.002317,0.004056
...,...,...,...,...,...,...
27353,0.000000,0.000000,0.000000,51,0.002607,0.000290
27354,0.000000,0.000000,0.000000,27,0.002607,0.000290
27355,0.000000,0.000000,0.000000,207,0.002607,0.002317
27356,0.000000,0.000000,0.000000,27,0.002607,0.003476


In [76]:
test.iloc[:,4:]

,aa,ra,jc,pa,dc_A,dc_B
0,0.000000,0.000000,0.00,40,0.002317,0.001448
1,0.000000,0.000000,0.00,24,0.002317,0.000869
2,0.000000,0.000000,0.00,432,0.004635,0.007822
3,1.242670,0.400000,0.10,57,0.005504,0.000869
4,0.000000,0.000000,0.00,391,0.004925,0.006663
...,...,...,...,...,...,...
1995,0.000000,0.000000,0.00,2,0.000290,0.000579
1996,0.000000,0.000000,0.00,2,0.000579,0.000290
1997,0.000000,0.000000,0.00,42,0.000579,0.006083
1998,1.508374,0.404053,0.05,236,0.017092,0.001159


In [78]:
y = [True if i < df.shape[0]/2 else False for i in range(df.shape[0])]
x = df.iloc[:,4:13]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.4)

In [191]:
collections.Counter(y)

Counter({True: 13679, False: 13679})

In [192]:
#from sklearn.preprocessing import StandardScaler
#normal = StandardScaler()
#x_train = normal.fit_transform(x_train)
#x_test = normal.transform(x_test)
#l1', 'C': 1.0, 'max_iter': 200, 'solver': 'liblinear'
LR = LogisticRegression(n_jobs = 4, penalty = 'l1', solver = 'liblinear').fit(df.iloc[:,4:-1], df.label)
LRT = LR.predict(dev.iloc[:,4:-1])

print(metrics.classification_report(dev.label, LRT))


              precision    recall  f1-score   support

           0       0.73      0.96      0.83      5055
           1       0.94      0.64      0.76      5055

    accuracy                           0.80     10110
   macro avg       0.83      0.80      0.80     10110
weighted avg       0.83      0.80      0.80     10110



C:\Users\Melissa Speer\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1353: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  warnings.warn("'n_jobs' > 1 does not have any effect when"


In [193]:
probs = LR.predict_proba(dev.iloc[:,4:-1])

In [194]:
metrics.roc_auc_score(dev.label, probs[:, 1])

0.8090631344038525

In [195]:
import collections
collections.Counter(LR.predict(test.iloc[:,4:]))

Counter({0: 1406, 1: 594})

In [196]:
results = LR.predict_proba(test.iloc[:,4:])
results[:,1]

array([0.06686987, 0.06587346, 0.09605604, ..., 0.0669954 , 1.        ,
       0.06531907])

In [278]:
output = pd.DataFrame({'Id': [i for i in range(1,2001)], 'Predicted': results[:,1]})
output.to_csv('logReg.csv', header = True, index= False)

### Handmade gridsearch

In [353]:
import itertools as it
from sklearn.metrics import accuracy_score
LR = LogisticRegression
LR_param = {'penalty':['l1', 'l2', 'elasticnet'],
            'C':[1.0,0.5,0.1,0.01], 'max_iter':[200],
           'solver':['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}
models = [('Logistic Regression',LR,LR_param)]

result = []
best = {}
for name, m, params in models:
    keys, values = zip(*params.items())
    permutations = [dict(zip(keys, v)) for v in it.product(*values)]
    
    for p in permutations:
        try:
            mod = m(**p)
            mod.fit(x_train, y_train);
            y_pred = mod.predict(x_dev)
            Acc_scored = accuracy_score(y_dev, y_pred)
            cnt = collections.Counter(mod.predict(test.iloc[:,4:]))
            best[str(p)] = (Acc_scored, cnt)
            #print('Model: ',name, ' HyperParams: ',p,' Accuracy: ',Acc_scored)
        except:
            pass#print(p)

    #best_params = max(best, key = lambda k:best[k][0])
    #result.append((name, best_params, best[best_params][0], best[best_params][1]))  

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did 

In [354]:
[print(k,v) for k,v in best.items()]

{'penalty': 'l1', 'C': 1.0, 'max_iter': 200, 'solver': 'liblinear'} (0.7816666666666666, Counter({0: 1439, 1: 561}))
{'penalty': 'l1', 'C': 1.0, 'max_iter': 200, 'solver': 'saga'} (0.5193333333333333, Counter({1: 1810, 0: 190}))
{'penalty': 'l1', 'C': 0.5, 'max_iter': 200, 'solver': 'liblinear'} (0.7816666666666666, Counter({0: 1439, 1: 561}))
{'penalty': 'l1', 'C': 0.5, 'max_iter': 200, 'solver': 'saga'} (0.5193333333333333, Counter({1: 1810, 0: 190}))
{'penalty': 'l1', 'C': 0.1, 'max_iter': 200, 'solver': 'liblinear'} (0.7815833333333333, Counter({0: 1439, 1: 561}))
{'penalty': 'l1', 'C': 0.1, 'max_iter': 200, 'solver': 'saga'} (0.5193333333333333, Counter({1: 1810, 0: 190}))
{'penalty': 'l1', 'C': 0.01, 'max_iter': 200, 'solver': 'liblinear'} (0.7808333333333334, Counter({0: 1440, 1: 560}))
{'penalty': 'l1', 'C': 0.01, 'max_iter': 200, 'solver': 'saga'} (0.51925, Counter({1: 1810, 0: 190}))
{'penalty': 'l2', 'C': 1.0, 'max_iter': 200, 'solver': 'newton-cg'} (0.7818333333333334, Coun

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

### LGBM

In [359]:
import lightgbm as lgbm

train_data = lgbm.Dataset(x_train, pd.Series(y_train))
test_data = lgbm.Dataset(x_dev, pd.Series(y_dev))

# define parameters
parameters = {
    'objective': 'binary',
    'metric': 'auc',
    'is_unbalance': 'true',
    'feature_fraction': 0.5,
    'bagging_fraction': 0.5,
    'bagging_freq': 20,
    'num_threads' : 2,
    'seed' : 76
}

# train lightGBM model
model = lgbm.train(parameters,
                   train_data,
                   valid_sets=test_data,
                   num_boost_round=1000,
                   early_stopping_rounds=40)

[LightGBM] [Info] Number of positive: 12810, number of negative: 12810
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000102 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1018
[LightGBM] [Info] Number of data points in the train set: 25620, number of used features: 4
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	valid_0's auc: 0.78175
Training until validation scores don't improve for 40 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2]	valid_0's auc: 0.78175
[3]	valid_0's auc: 0.822314
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4]	valid_0's auc: 0.822314
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5]	valid_0's auc: 0.822

In [326]:
predict

array([0.49348902, 0.49348902, 0.49348902, ..., 0.49348902, 0.49348902,
       0.49348902])

In [328]:
predict= model.predict(test.iloc[:,4:])
collections.Counter(["pos" if i>0.5 else "neg" for i in predict])

Counter({'neg': 1439, 'pos': 561})

### Support Vector Machine (SVM)

In [197]:
SVM = LinearSVC(max_iter=10000)
#SVM_predict = SVM.predict(x_test)
clf = CalibratedClassifierCV(SVM) 
clf.fit(x_train, y_train)
clf_predict = clf.predict(x_dev)


print(metrics.classification_report(y_dev, clf_predict))

C:\Users\Melissa Speer\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\Melissa Speer\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\Melissa Speer\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\Melissa Speer\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


              precision    recall  f1-score   support

           0       0.66      0.99      0.79      5055
           1       0.98      0.48      0.65      5055

    accuracy                           0.74     10110
   macro avg       0.82      0.74      0.72     10110
weighted avg       0.82      0.74      0.72     10110



C:\Users\Melissa Speer\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [198]:
probs = clf.predict_proba(x_dev)

In [199]:
probs

array([[0.01230938, 0.98769062],
       [0.00379426, 0.99620574],
       [0.00392456, 0.99607544],
       ...,
       [0.82891899, 0.17108101],
       [0.82907605, 0.17092395],
       [0.82921991, 0.17078009]])

In [200]:
metrics.roc_auc_score(y_dev, probs[:, 1])

0.8188207267045682

In [201]:
collections.Counter(clf.predict(test.iloc[:,4:]))

Counter({0: 1611, 1: 389})

In [202]:
collections.Counter(y_test)

Counter({False: 5476, True: 5468})

In [203]:
results = clf.predict_proba(test.iloc[:,4:])
results[:,1]

array([0.17055957, 0.17083283, 0.164178  , ..., 0.17049082, 0.8812935 ,
       0.17092608])

In [160]:
output = pd.DataFrame({'Id': [i for i in range(1,2001)], 'Predicted': predict})
output.to_csv('lightGBM.csv', header = True, index= False)

## Random Forest Classifier

In [181]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(max_depth=12, random_state=0, oob_score=True, bootstrap=True).fit(x_train, y_train) # max_depth 6 is optimal depth.
rfc_pred = rfc.predict(x_dev)

print(metrics.classification_report(y_dev, rfc_pred))

              precision    recall  f1-score   support

           0       0.86      0.94      0.90      5055
           1       0.94      0.84      0.89      5055

    accuracy                           0.89     10110
   macro avg       0.90      0.89      0.89     10110
weighted avg       0.90      0.89      0.89     10110



In [176]:
print('Score: ', rfc.score(x_train, y_train))

Score:  0.9922874479128592


In [177]:
print(rfc.oob_score_)

0.9812486292857665


In [182]:
probs = rfc.predict_proba(x_dev)

In [183]:
probs

array([[0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       ...,
       [0.2225993 , 0.7774007 ],
       [0.98262554, 0.01737446],
       [0.94112617, 0.05887383]])

In [184]:
metrics.roc_auc_score(y_dev, probs[:, 1])

0.9645049852219063

In [185]:
collections.Counter(rfc.predict(test.iloc[:,4:]))

Counter({0: 843, 1: 1157})

In [173]:
results = rfc.predict_proba(test.iloc[:,4:])
results[:,1]

array([0.10305407, 0.06810847, 0.63921827, ..., 0.10767577, 1.        ,
       0.29842463])

In [174]:
output = pd.DataFrame({'Id': [i for i in range(1,2001)], 'Predicted': results[:,1]})
output.to_csv('randomForest_6feat.csv', header = True, index= False)